The following command installs our library

In [1]:
!pip install git+https://github.com/sarah-keren/ai_dm


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/sarah-keren/ai_dm to /tmp/pip-req-build-oklil_xe
  Running command git clone -q https://github.com/sarah-keren/ai_dm /tmp/pip-req-build-oklil_xe
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 21.8 MB 1.5 MB/s 
  Created wheel for AI-agents: filename=AI_agents-0.0.0-py3-none-any.whl size=29830 sha256=21bb344c3d439dd46c8275a63aeecd01e798186d13b5e77cb9866cda1b823462
  Stored in directory: /tmp/pip-ephem-wheel-cache-b233138q/wheels/67/17/23/0dd07f059863e14f5daab572805690194ad5e79b0a0291849e
Successfully built AI-agents


In [2]:
!pip install gym


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


We will be working with Open Ai's Taxi domain that looks like this


In [3]:
import gym

env = gym.make("Taxi-v3", render_mode='ansi').env
env.reset ()
print(env.render())

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+




/usr/local/lib/python3.7/dist-packages/gym/core.py:52: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  "The argument mode in render method is deprecated; "


The filled square represents the taxi, which is yellow without a passenger and green with a passenger. The pipes ("|") represent walls which the taxi cannot traverse. R, G, Y, B are the possible pickup and destination locations. The passenger can also be in the taxi. The blue letter represents the current passenger pick-up location, and the purple letter is the current destination.

The actions space is:

0 = south 1 = north 2 = east 3 = west 4 = pickup 5 = dropoff

The state encoding is as follows (play around with the values)

In [4]:
state = env.encode(1, 3, 4, 1) # (taxi row, taxi column, passenger index, destination index)
print("State:", state)

print(env.render())
env.unwrapped.s = state
print(env.render())

State: 177
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+


+---------+
|R: | : :G|
| : | :_: |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+




Now let's import the GymProblem which will wrap the Taxi domain and the search methods we want to apply

In [5]:
import gym
from ai_dm.Environments.gym_envs.gym_problem import GymProblem
from ai_dm.Search.best_first_search import best_first_search, breadth_first_search, depth_first_search, a_star, depth_first_search_l
import ai_dm.Search.utils as utils
import ai_dm.Search.defs as defs
import ai_dm.Search.heuristic as heuristic




The following will create and recreate the taxi environment we want to explore

In [6]:
def create_env():

    # define the environment
    taxi_env = gym.make("Taxi-v3", render_mode='ansi').env
    taxi_env.reset()
    init_state = taxi_env.encode(0, 3, 4, 1)  # (taxi row, taxi column, passenger index, destination index)
    taxi_row, taxi_col, pass_idx, dest_idx = taxi_env.decode(init_state)
    print(taxi_row)
    taxi_env.unwrapped.s = init_state
    print("State:", init_state)
    print(env.render())
    return taxi_env   


And now let's run Breadth First Search


In [7]:
taxi_env = create_env()

# create a wrapper of the environment to the search
taxi_p = GymProblem(taxi_env, taxi_env.unwrapped.s)


# perform BFS
[best_value, best_node, best_plan, explored_count, ex_terminated] = breadth_first_search(problem=taxi_p,
                                                                                         log=True,
                                                                                         log_file=None,
                                                                                         iter_limit=defs.NA,
                                                                                         time_limit=defs.NA,
                                                                                        )


print(best_plan)
for action_id in best_plan:
    print(taxi_env.step(int(action_id)))
    taxi_p.apply_action(action_id)
    print(taxi_p.env.render())


0
State: 77
+---------+
|R: | : :G|
| : | :_: |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+


best_first_design(node) node number 1 cur_node:<Node 77>
InMethod best_first_design(node): explored_count:1 cur_node:<Node 77>, node_eval_time:0.000
best_first_design(node) node number 2 cur_node:<Node 177>
InMethod best_first_design(node): explored_count:2 cur_node:<Node 177>, node_eval_time:0.000
best_first_design(node) node number 3 cur_node:<Node 97>
InMethod best_first_design(node): explored_count:3 cur_node:<Node 97>, node_eval_time:0.000
best_first_design(node) node number 4 cur_node:<Node 57>
InMethod best_first_design(node): explored_count:4 cur_node:<Node 57>, node_eval_time:0.000
best_first_design(node) node number 5 cur_node:<Node 277>
InMethod best_first_design(node): explored_count:5 cur_node:<Node 277>, node_eval_time:0.000
best_first_design(node) node number 6 cur_node:<Node 197>
InMethod best_first_design(node): explored_count:6 cur_node:<Node 197>, node_eval_time:0.000
be

Let's see the same methods, but now with the explicit call to BestFirstSearch (which shows us how Breadth First Search is a special case of BestFirstSearch)

In [8]:
taxi_env = create_env()

# create a wrapper of the environment to the search
taxi_p = GymProblem(taxi_env, taxi_env.unwrapped.s)

# perform BFS
[best_value, best_node, best_plan, explored_count, ex_terminated] = best_first_search(problem=taxi_p,
                                                                                      frontier=utils.FIFOQueue(),
                                                                                      closed_list=utils.ClosedListOfKeys(),
                                                                                      termination_criteria=utils.TerminationCriteriaGoalStateReached(),
                                                                                      evaluation_criteria=utils.EvaluationCriteriaGoalCondition(),
                                                                                      prune_func=None,
                                                                                      log=True, log_file=None,
                                                                                      iter_limit=defs.NA,
                                                                                      time_limit=defs.NA,
                                                                                      )
print(best_plan)
for action_id in best_plan:
    taxi_p.apply_action(action_id)
    print(env.render())

0
State: 77
+---------+
|R: | : :G|
| : | :_: |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+


best_first_design(node) node number 1 cur_node:<Node 77>
InMethod best_first_design(node): explored_count:1 cur_node:<Node 77>, node_eval_time:0.000
best_first_design(node) node number 2 cur_node:<Node 177>
InMethod best_first_design(node): explored_count:2 cur_node:<Node 177>, node_eval_time:0.000
best_first_design(node) node number 3 cur_node:<Node 97>
InMethod best_first_design(node): explored_count:3 cur_node:<Node 97>, node_eval_time:0.000
best_first_design(node) node number 4 cur_node:<Node 57>
InMethod best_first_design(node): explored_count:4 cur_node:<Node 57>, node_eval_time:0.000
best_first_design(node) node number 5 cur_node:<Node 277>
InMethod best_first_design(node): explored_count:5 cur_node:<Node 277>, node_eval_time:0.000
best_first_design(node) node number 6 cur_node:<Node 197>
InMethod best_first_design(node): explored_count:6 cur_node:<Node 197>, node_eval_time:0.000
be

**And** now, let's run DFS

In [9]:
taxi_env = create_env()
# create a wrapper of the environment to the search
taxi_p = GymProblem(taxi_env, taxi_env.unwrapped.s)

# perform DFS
[best_value, best_node, best_plan, explored_count, ex_terminated] = depth_first_search(problem=taxi_p,
                                                                                       log=True,
                                                                                       log_file=None,
                                                                                       iter_limit=defs.NA,
                                                                                       time_limit=defs.NA,
                                                                                       )

print(best_plan)
for action_id in best_plan:
    taxi_p.apply_action(action_id)
    print(env.render())

0
State: 77
+---------+
|R: | : :G|
| : | :_: |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+


best_first_design(node) node number 1 cur_node:<Node 77>
InMethod best_first_design(node): explored_count:1 cur_node:<Node 77>, node_eval_time:0.000
best_first_design(node) node number 2 cur_node:<Node 57>
InMethod best_first_design(node): explored_count:2 cur_node:<Node 57>, node_eval_time:0.000
best_first_design(node) node number 3 cur_node:<Node 157>
InMethod best_first_design(node): explored_count:3 cur_node:<Node 157>, node_eval_time:0.000
best_first_design(node) node number 4 cur_node:<Node 257>
InMethod best_first_design(node): explored_count:4 cur_node:<Node 257>, node_eval_time:0.000
best_first_design(node) node number 5 cur_node:<Node 237>
InMethod best_first_design(node): explored_count:5 cur_node:<Node 237>, node_eval_time:0.000
best_first_design(node) node number 6 cur_node:<Node 217>
InMethod best_first_design(node): explored_count:6 cur_node:<Node 217>, node_eval_time:0.000


and now let's run DFS-L

In [10]:
taxi_env = create_env()
# create a wrapper of the environment to the search
taxi_p = GymProblem(taxi_env, taxi_env.unwrapped.s)

# perform DFS
[best_value, best_node, best_plan, explored_count, ex_terminated] = depth_first_search_l(problem=taxi_p,
                                                                                       max_depth=2,
                                                                                       log=True,
                                                                                       log_file=None,
                                                                                       iter_limit=defs.NA,
                                                                                       time_limit=defs.NA,
                                                                                       )

print(best_plan)
for action_id in best_plan:
    taxi_p.apply_action(action_id)
    print(env.render())

0
State: 77
+---------+
|R: | : :G|
| : | :_: |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+


best_first_design(node) node number 1 cur_node:<Node 77>
InMethod best_first_design(node): explored_count:1 cur_node:<Node 77>, node_eval_time:0.000
best_first_design(node) node number 2 cur_node:<Node 57>
InMethod best_first_design(node): explored_count:2 cur_node:<Node 57>, node_eval_time:0.000
best_first_design(node) node number 3 cur_node:<Node 97>
InMethod best_first_design(node): explored_count:3 cur_node:<Node 97>, node_eval_time:0.000
best_first_design(node) node number 4 cur_node:<Node 177>
InMethod best_first_design(node): explored_count:4 cur_node:<Node 177>, node_eval_time:0.000
solution is: []
[]


Finally, let's see how A-Star works on the same problem (with a ridiculously simple heuristic that gives 0 to terminal nodes and 1 to all other nodes):

In [11]:
taxi_env = create_env()

# create a wrapper of the environment to the search
taxi_p = GymProblem(taxi_env, taxi_env.unwrapped.s)

# perform A*
[best_value, best_node, best_plan, explored_count, ex_terminated] = best_first_search(problem=taxi_p,
                                                                                      frontier=utils.PriorityQueue(heuristic.goal_heuristic),
                                                                                      closed_list=utils.ClosedListOfKeys(),
                                                                                      termination_criteria=utils.TerminationCriteriaGoalStateReached(),
                                                                                      evaluation_criteria=utils.EvaluationCriteriaGoalCondition(),
                                                                                      prune_func=None,
                                                                                      log=True, 
                                                                                      log_file=None,
                                                                                      iter_limit=defs.NA,
                                                                                      time_limit=defs.NA,
                                                                                      )

print(best_plan)
for action_id in best_plan:
    taxi_p.apply_action(action_id)
    print(env.render())

0
State: 77
+---------+
|R: | : :G|
| : | :_: |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+


best_first_design(node) node number 1 cur_node:<Node 77>
InMethod best_first_design(node): explored_count:1 cur_node:<Node 77>, node_eval_time:0.000
best_first_design(node) node number 2 cur_node:<Node 57>
InMethod best_first_design(node): explored_count:2 cur_node:<Node 57>, node_eval_time:0.000
best_first_design(node) node number 3 cur_node:<Node 97>
InMethod best_first_design(node): explored_count:3 cur_node:<Node 97>, node_eval_time:0.000
best_first_design(node) node number 4 cur_node:<Node 85>
InMethod best_first_design(node): explored_count:4 cur_node:<Node 85>, node_eval_time:0.000
solution is: ['2', '5']
['2', '5']
+---------+
|R: | : :G|
| : | :_: |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+


+---------+
|R: | : :G|
| : | :_: |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+




Now it's your time to suggest new algorithms..